# Lab Part 4
# Neural Network Model Template

#
#



## Part 1: Define where we store our data
This is pretty simple our folder structure looks like this
- data/
    - train/
    - test/

So we simply create 2 variables to store these locations

In [2]:
data_dir = 'data/' # change this to your data directory
train_dir = data_dir + 'train/' # directory for training images
test_dir = data_dir + 'test/' # directory for test images we use the test images to make sure our model is working well

## Part 2: Loading in the data
### Dealing with images
Our image files that we are using for training are pretty large. Your computer may crash if we loaded them all at once so instead we will just store where the images are stored. Later you will see how we can automatically load these files when we run our code. One thing to note is that each image file is named (somenumber).jpg that number is the number of milliseconds since 1 January 1970 (Unix Time) when the file was recorded. This means that we will never have any overlapping files. Unfortunately this does mean we will have to sort the times from smallest to largest to make sure that they correspond with the steering angles.

### Dealing with steering angles
We store our steering values as a csv (comma separated values) file. Python has a built in csv module so we can easily use it to load in all the steering values into a list. These don't take up much ram so it doesn't matter if we load them all in at once.

### Making sure the files line up
Since we sort our images, and the csv file is read in line by line the files will line up with their correct steering angles. It looks something like this. If we mess this up then the data being fed into your network would be totally incoherent. Garbage in Garbage out.

train_img =   | image 1    | image 2    | image 3    | image 4    | image 5    |...

train_steer = | steering 1 | steering 2 | steering 3 | steering 4 | steering 5 |...

In [3]:
import csv
import os
# lets make a function that loads the images and labels
def load_data(directory):
    image__paths = []
    csv_file = ""
    for file in os.listdir(directory): # for each file in the directory
        if file.endswith(".jpg"): # if the file is an image
            image__paths.append(directory + file) # add the image path to the list
        if file.endswith(".csv"): # if the file is a csv file
            csv_file = file # we save it for later

    # now our files are in the train list we need to sort them from smallest file name to largest. The file name is the exact time the image was taken.
    image__paths.sort(key=lambda x: int(x.split('/')[-1][:-4])) # the lambda function returns the numbers in the file name

    # now we need to read the csv file and get the steering angles
    with open(directory + csv_file, 'r') as f:
        reader = csv.reader(f) # create a reader object
        steering_angles = [] # create a list to store the steering angles
        for row in reader: # for each row in the csv file
            steering_angles.append(float(row[0])) # add the steering angle to the list
    return image__paths, steering_angles # return the image paths and steering angles

In [24]:
train_img, train_steer = load_data(train_dir) # load the training data

## Part 3: Loading the images on the fly

Now we create a data generator that automatically fetches the images when we need it.
This generator will also apply our image processing that we learnt about in our last lab.
This generator takes in a batch_size which means we can increase or decrease the amount of data returned based on how much RAM our computer has.

In [25]:
import numpy as np
import cv2 as cv

def batch_generator(image_paths, steering_angles, batch_size):
    while True:
        for start in range(0, len(image_paths), batch_size):
            x_batch = []
            y_batch = []
            end = min(start + batch_size, len(image_paths))
            ids_batch = image_paths[start:end]
            for id in ids_batch:
                img = cv.imread(id)
                img = cv.resize(img, (100, 66))
                x_batch.append(img)
                y_batch.append(steering_angles[image_paths.index(id)])
            x_batch = np.array(x_batch, np.float32)
            y_batch = np.array(y_batch, np.float32)
            yield x_batch, y_batch


## Part 4: This is where you will create your model
try modifying the layers to try to get the lowest mse score,
This is going to require some trial and error so make sure you spend some time experimenting.

In [30]:
import keras as ks
from keras.models import Sequential
from keras.layers import Dense, InputLayer

# a sequential model is a model that is made up of layers
model = Sequential()
# the input layer is the first layer in the model
model.add(InputLayer(input_shape=(100, 66, 3)))

# try modifying the number of nodes in the hidden layer to see how it affects the model
# you can also try changing the activation function to see how it affects the model
# adding more layers to the model may also help
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1 ,activation='linear'))

batch_size = 32 

model.compile(optimizer='adam', loss='mse')
model.fit(batch_generator(train_img, train_steer, batch_size), steps_per_epoch=np.ceil(float(len(train_img)) / float(batch_size)), epochs=25) # changing the number of epochs may help the model!

Epoch 1/25
5/5 [==============================] - 1s 67ms/step - loss: 10489.0049
Epoch 2/25
5/5 [==============================] - 0s 66ms/step - loss: 8783.3252
Epoch 3/25
5/5 [==============================] - 0s 66ms/step - loss: 7340.7886
Epoch 4/25
5/5 [==============================] - 0s 69ms/step - loss: 6068.4614
Epoch 5/25
5/5 [==============================] - 0s 69ms/step - loss: 5009.4741
Epoch 6/25
5/5 [==============================] - 0s 66ms/step - loss: 4129.6631
Epoch 7/25
5/5 [==============================] - 0s 67ms/step - loss: 3374.8501
Epoch 8/25
5/5 [==============================] - 0s 69ms/step - loss: 2797.7703
Epoch 9/25
5/5 [==============================] - 0s 66ms/step - loss: 2465.3472
Epoch 10/25
5/5 [==============================] - 0s 65ms/step - loss: 2349.3149
Epoch 11/25
5/5 [==============================] - 0s 65ms/step - loss: 2359.5649
Epoch 12/25
5/5 [==============================] - 0s 66ms/step - loss: 2395.0208
Epoch 13/25
5/5 [=======